#TODO
- Decide Epochs
- GAther y predict target 
- Test submission to y predict target 

In [175]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import scipy.stats as stats
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
# data = None
# test_data = None
# import os
# for dirname, _, filenames in os.walk('/kaggle/input/'):
#     for filename in filenames:
#         path = os.path.join(dirname, filename)
#         if filename == 'train.csv':
#             data = pd.read_csv(path)
#         if filename == 'test.csv':
#             test_data = pd.read_csv(path)
#         if filename == 'store.csv':
#             store_data = pd.read_csv(path)
        

In [176]:
from google.colab import drive #mounting to drive
drive.mount('/content/drive', force_remount=True)

# Reading Zip
from zipfile import ZipFile
file_name = '/content/drive/My Drive/CE889/Group_Assignment/rossmann-store-sales.zip' 
with ZipFile(file_name, 'r') as zip:
    zip.printdir()
    zip.extractall()
    print('Done')

store_data = pd.read_csv('store.csv')

#Assigning data set to df
data = pd.read_csv('train.csv')

#Assigning Test data to df
test_data = pd.read_csv('test.csv')

Mounted at /content/drive
File Name                                             Modified             Size
sample_submission.csv                          2019-12-11 03:57:00       317611
store.csv                                      2019-12-11 03:57:00        45010
test.csv                                       2019-12-11 03:57:00      1427425
train.csv                                      2019-12-11 03:57:02     38057952
Done


/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [177]:
# Join train set and store set, test set and store set, inner refers to using the common keys
# on refers to the label
new_train = data.merge(store_data, how='inner', on='Store')
new_test = test_data.merge(store_data, how='inner', on='Store')

In [178]:
new_train['Date'] = pd.to_datetime(new_train['Date'])
new_train['Year'] = new_train.Date.dt.year
new_train['Month'] = new_train.Date.dt.month
new_train['Day'] = new_train.Date.dt.day

new_test['Date'] = pd.to_datetime(new_test['Date'])
new_test['Year'] = new_train.Date.dt.year
new_test['Month'] = new_test.Date.dt.month
new_test['Day'] = new_test.Date.dt.day

In [179]:
new_train["CompetitionDistance"]=new_train["CompetitionDistance"].fillna(99999)
new_test["CompetitionDistance"]=new_test["CompetitionDistance"].fillna(99999)

In [180]:
 # convert assortment and storetype and stateholiday in training data to int
cleanup = {"Assortment": {"a":0, "b":1, "c":2}}
new_train.replace(cleanup, inplace=True)
new_train['Assortment'] = new_train['Assortment'].astype(int)

cleanup2 = {"StoreType": {"a":0, "b":1, "c":2, "d":3}}
new_train.replace(cleanup2, inplace=True)
new_train['StoreType'] = new_train['StoreType'].astype(int)

cleanup3 = {"StateHoliday": {"a":0, "b":1, "c":2}}
new_train.replace(cleanup3, inplace=True)
new_train['StateHoliday'] = new_train['StateHoliday'].astype(int)

In [181]:
 # convert assortment and storetype in testing data to int
cleanup = {"Assortment": {"a":0, "b":1, "c":2}}
new_test.replace(cleanup, inplace=True)
new_test['Assortment'] = new_test['Assortment'].astype(int)

cleanup2 = {"StoreType": {"a":0, "b":1, "c":2, "d":3}}
new_test.replace(cleanup2, inplace=True)
new_test['StoreType'] = new_test['StoreType'].astype(int)

cleanup3 = {"StateHoliday": {"a":0, "b":1, "c":2}}
new_train.replace(cleanup3, inplace=True)
new_train['StateHoliday'] = new_train['StateHoliday'].astype(int)

In [182]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1017209 entries, 0 to 1017208
Data columns (total 21 columns):
 #   Column                     Non-Null Count    Dtype         
---  ------                     --------------    -----         
 0   Store                      1017209 non-null  int64         
 1   DayOfWeek                  1017209 non-null  int64         
 2   Date                       1017209 non-null  datetime64[ns]
 3   Sales                      1017209 non-null  int64         
 4   Customers                  1017209 non-null  int64         
 5   Open                       1017209 non-null  int64         
 6   Promo                      1017209 non-null  int64         
 7   StateHoliday               1017209 non-null  int64         
 8   SchoolHoliday              1017209 non-null  int64         
 9   StoreType                  1017209 non-null  int64         
 10  Assortment                 1017209 non-null  int64         
 11  CompetitionDistance        1017209 no

In [183]:
 # Null value exists in open, fill with one, cuz stores would open except from holidays
new_train['Open'] = new_train['Open'].fillna(1)
new_train['Open'] = new_train['Open'].astype(int)


new_test['Open'] = new_test['Open'].fillna(1)
new_test['Open' ]= new_test['Open'].astype(int)

In [184]:
 # fill nan with month 0, years in the far future
new_train['CompetitionOpenSinceMonth'].fillna(1, inplace=True)
new_train['CompetitionOpenSinceYear'].fillna(2100, inplace=True)

new_train['Promo2SinceWeek'].fillna(1, inplace=True)
new_train['Promo2SinceYear'].fillna(2100, inplace=True)
new_train['PromoInterval'].fillna(0, inplace=True)

new_test['CompetitionOpenSinceMonth'].fillna(1, inplace=True)
new_test['CompetitionOpenSinceYear'].fillna(2100, inplace=True)

new_test['Promo2SinceWeek'].fillna(1, inplace=True)
new_test['Promo2SinceYear'].fillna(2100, inplace=True)
new_test['PromoInterval'].fillna(0, inplace=True)

In [185]:
 # combine competition month and year and promo week year to form days
new_train['CompetitionDays'] = 365 * (new_train['Year'] - new_train['CompetitionOpenSinceYear']) + new_train['Month'] - new_train['CompetitionOpenSinceMonth']

new_train['PromoOpenDays'] = 365 * (new_train['Year'] - new_train['Promo2SinceYear']) + ((new_train['Day'] - 7 * new_train['Promo2SinceWeek']))

new_test['CompetitionDays'] = 365 * (new_test['Year'] - new_test['CompetitionOpenSinceYear']) + new_test['Month'] - new_test['CompetitionOpenSinceMonth']

new_test['PromoOpenDays'] = 365 * (new_test['Year'] - new_test['Promo2SinceYear']) + ((new_test['Day'] - 7 * new_test['Promo2SinceWeek']))

In [186]:
new_train.drop(columns = ['CompetitionOpenSinceMonth'], inplace = True)
new_train.drop(columns =['CompetitionOpenSinceYear'], inplace = True)
new_train.drop(columns =['Promo2SinceWeek'], inplace = True)
new_train.drop(columns =['Promo2SinceYear'], inplace = True)
new_train.drop(columns =['Date'], inplace = True)

In [187]:
new_train = new_train.query('Open==1')
new_test = new_test.query('Open==1')

In [188]:
# Normalize
"""
new_train_plus = new_train
columns = ['Sales', 'CompetitionDistance', 'CompetitionDays', 'PromoOpenDays', "Customers"]
for col in columns:
    max_value = np.max(new_train_plus[col])
    min_value = np.min(new_train_plus[col])
    new_train_plus[col] = (new_train_plus[col] - min_value) / (max_value - min_value)

new_train = new_train_plus
"""
new_train.drop(columns=['Customers'],inplace = True)


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [189]:
new_train.dtypes

Store                    int64
DayOfWeek                int64
Sales                    int64
Open                     int64
Promo                    int64
StateHoliday             int64
SchoolHoliday            int64
StoreType                int64
Assortment               int64
CompetitionDistance    float64
Promo2                   int64
PromoInterval           object
Year                     int64
Month                    int64
Day                      int64
CompetitionDays        float64
PromoOpenDays          float64
dtype: object

In [190]:
new_train = pd.get_dummies(new_train, columns = ['PromoInterval'])
new_test = pd.get_dummies(new_test, columns = ['PromoInterval'])

In [191]:
new_test.shape

(35104, 25)

In [192]:
x = new_train.drop(["Sales"], axis = 1)
y = new_train["Sales"]

In [193]:
x = x.to_numpy()
y = y.to_numpy()

In [194]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Scaling the data 
scaler = MinMaxScaler()
scaler.fit_transform(x,y)

array([[0.        , 0.66666667, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.5       , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.33333333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.66666667, 0.        , ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.5       , 0.        , ..., 0.        , 0.        ,
        1.        ],
       [1.        , 0.33333333, 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [195]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [196]:
# X_train.shape, X_test.shape, y_test.shape, y_train.shape

In [197]:
# def df_to_X_y(df, window_size=5):
#   df_as_np = df.to_numpy()
#   X = []
#   y = []
#   for i in range(len(df_as_np)-window_size):
#     row = [[a] for a in df_as_np[i:i+window_size]]
#     X.append(row)
#     label = df_as_np[i+window_size]
#     y.append(label)
#   return np.array(x), np.array(y)

In [198]:
# WINDOW_SIZE = 5
# X1, y1 = df_to_X_y(x, WINDOW_SIZE)
# X1.shape, y1.shape

In [199]:
# x.shape

In [200]:
# def create_timestep(i, window_size):
  
#       nrows = i.shape[0] - window_size + 1
#       p,q = i.shape
#       m,n = i.strides
#       strided = np.lib.stride_tricks.as_strided
#       output_X = strided(i,shape=(nrows,window_size,q),strides=(m,m,n))
  
#       return output_X
# ''' 
# Reshaping an array to give it (n, n, n)
# is necessary before feeding into a RNN

# window_size --> passes in the current iteration, as well as n previous layers.

# window_size is used for noisy or erroneous data, the window is used to mitigate

# window_size can be defined from (1-->n)

# '''

# # create_timestep(input, window_size)

In [201]:
# X_train = create_timestep(X_train, 5)

In [202]:
# X_test = create_timestep(X_test, 5)

In [203]:
X_train.shape, X_test.shape, y_test.shape, y_train.shape

((675513, 19), (168879, 19), (168879,), (675513,))

In [204]:
# Resizing Shape of test and train set to 3d arrays , needs to be changed depending on size of train test split and if data changes
# The size of shapes should reflect the split data 
X_train = X_train.reshape(675513, 1, 19)
X_test = X_test.reshape(168879, 1, 19)

In [205]:
X_train.shape, X_test.shape, y_test.shape, y_train.shape

((675513, 1, 19), (168879, 1, 19), (168879,), (675513,))

#LSTM ATTEMPT 2


In [206]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

model1 = Sequential()
model1.add(InputLayer((1, 19)))
model1.add(LSTM(64))
model1.add(Dense(8, 'relu'))
model1.add(Dense(1, 'linear'))

model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 64)                21504     
                                                                 
 dense_8 (Dense)             (None, 8)                 520       
                                                                 
 dense_9 (Dense)             (None, 1)                 9         
                                                                 
Total params: 22,033
Trainable params: 22,033
Non-trainable params: 0
_________________________________________________________________


In [207]:
cp1 = ModelCheckpoint('model1/', save_best_only=True)
model1.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=1e-4), metrics=[RootMeanSquaredError()])

In [ ]:
model1.fit(X_train, y_train, epochs=3, callbacks=[cp1])

Epoch 1/3
21102/21110 [============================>.] - ETA: 0s - loss: 56257884.0000 - root_mean_squared_error: 7500.5254

21110/21110 [==============================] - 79s 4ms/step - loss: 56256960.0000 - root_mean_squared_error: 7500.4639
Epoch 2/3
21106/21110 [============================>.] - ETA: 0s - loss: 47289180.0000 - root_mean_squared_error: 6876.7129

21110/21110 [==============================] - 76s 4ms/step - loss: 47286896.0000 - root_mean_squared_error: 6876.5469
Epoch 3/3
 6019/21110 [=======>......................] - ETA: 56s - loss: 38907172.0000 - root_mean_squared_error: 6237.5615

In [ ]:
# from tensorflow.keras.models import load_model
# model1 = load_model('model1/')

In [ ]:
# train_predictions = model1.predict(X_train).flatten()
# train_results = pd.DataFrame(data={'Train Predictions':train_predictions, 'Actuals':y_train})
# train_results

In [ ]:
# test_predictions = model1.predict(X_test).flatten()
# test_results = pd.DataFrame(data={'Test Predictions':test_predictions, 'Actuals':y_test})
# test_results


In [ ]:
# def feedforward(models: [], x):
    
#     output = torch.tensor(x).float()

#     for model in models:
#         output1 = model(output)
#         output = output1
        
#     return output

In [ ]:
for test in new_test:
    exist = False
    for train in new_train:
        if train == test:
            exist = True
    
    if not exist:
        new_test = new_test.drop([test], axis=1)

print(len(new_test.dtypes))
print(len(new_train.dtypes)-1) # beacause Sales is y data

In [ ]:
cleanup3 = {"StateHoliday": {"a":0, "b":1, "c":2}}
new_test.replace(cleanup3, inplace=True)
new_test['StateHoliday'] = new_test['StateHoliday'].astype(int)

In [ ]:
# import tensorflow as tf
# def feedforward(model1, X_test):
    
#     output = tf.cast(
#     x, float, name=None
# )

#     for i in model1:
#         output1 = model1(output)
#         output = output1
        
#     return output

In [ ]:
# new_test.reshape((35104,1, 19)
# )

In [ ]:
test = new_test.to_numpy()
test = test.reshape((35104,1, 19)
)
test.shape

In [ ]:
# test = test.flatten()


In [ ]:
# test = new_test.to_numpy()
target = model1.predict(test).flatten()
# arr = target.data.cpu().numpy()
arr = target
prediction_data = pd.DataFrame(arr)
prediction_data

In [ ]:
# output = pd.DataFrame({'Id': test, 'Sales': prediction_data[0]})
# output.fillna(0, inplace=True)
# output.to_csv('submission.csv', index=False)
# output

In [ ]:
# test = new_test.to_numpy()
# target = feedforward(models, test)
# arr = target.data.cpu().numpy()
# prediction_data = pd.DataFrame(arr)
# prediction_data

In [ ]:
# test_results['Actuals'].median()

In [ ]:
# test_results['Test Predictions'].mean()

In [ ]:
# test = new_test.to_numpy()
# y = feedforward(models, test)
# arr = y.data.cpu().numpy()
# prediction_data = pd.DataFrame(arr)
# prediction_data

In [ ]:
# output = pd.DataFrame({'Id': test_data.Id, 'Sales': prediction_data[0]})
# output.fillna(0, inplace=True)
# output.to_csv('submission.csv', index=False)
# output


#LSTM ATTEMPT 1


In [ ]:
# # Imports
# import torch
# import torchvision  # torch package for vision related things
# import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
# import torchvision.datasets as datasets  # Standard datasets
# import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
# from torch import optim  # For optimizers like SGD, Adam, etc.
# from torch import nn  # All neural network modules
# from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
# from tqdm import tqdm  # For a nice progress bar!

# # Set device
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# # Hyperparameters
# input_size = 28
# hidden_size = 256
# num_layers = 2
# num_classes = 10
# sequence_length = 28
# learning_rate = 0.0001
# batch_size = 64
# num_epochs = 2




In [ ]:
# # Recurrent neural network (many-to-one)
# class RNN(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, num_classes):
#         super(RNN, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size * sequence_length, num_classes)

#     def forward(self, x):
#         # Set initial hidden and cell states
#         h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)

#         # Forward propagate LSTM
#         out, _ = self.rnn(x, h0)
#         out = out.reshape(out.shape[0], -1)

#         # Decode the hidden state of the last time step
#         out = self.fc(out)
#         return out

In [ ]:
# # Load Data
# train_dataset = x
# test_dataset = y
# train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

# # Initialize network (try out just using simple RNN, or GRU, and then compare with LSTM)
# model = RNN(input_size, hidden_size, num_layers, num_classes).to(device) # Need to change to RNN if using RNN

# # Loss and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # Train Network
# for epoch in range(num_epochs):
#     for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
#         # Get data to cuda if possible
#         data = data.to(device=device).squeeze(1)
#         targets = targets.to(device=device)

#         # forward
#         scores = model(data)
#         loss = criterion(scores, targets)

#         # backward
#         optimizer.zero_grad()
#         loss.backward()

#         # gradient descent update step/adam step
#         optimizer.step()

# # Check accuracy on training & test to see how good our model
# def check_accuracy(loader, model):
#     num_correct = 0
#     num_samples = 0

#     # Set model to eval
#     model.eval()

#     with torch.no_grad():
#         for x, y in loader:
#             x = x.to(device=device).squeeze(1)
#             y = y.to(device=device)

#             scores = model(x)
#             _, predictions = scores.max(1)
#             num_correct += (predictions == y).sum()
#             num_samples += predictions.size(0)

#     # Toggle model back to train
#     model.train()
#     return num_correct / num_samples


# print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
# print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")